<a href="https://colab.research.google.com/github/mshumer/gpt-prompt-engineer/blob/main/gpt_prompt_engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Prompt Engineering 
Original Notebook By Matt Shumer (https://twitter.com/mattshumer_)

Original Github repo: https://github.com/mshumer/gpt-prompt-engineer

In this notebook, we will simulate how to generate an optimal prompt/instruction for a given task. In this case, we will go over the Tealium dataset and generate optimal instructions for each question. This notebook is modified for the Tealium-AWS use case. 

## Installations

In [6]:
!pip install -r ../requirements.txt -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.1 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.


## Import Libraries

In [1]:
from datasets import load_dataset, Dataset
import pandas as pd
from ast import literal_eval
import sys
from pathlib import Path

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
path_to_src_code = '/home/ec2-user/SageMaker/Auto_Prompt'
sys.path.append(path_to_src_code)

In [3]:
from src.utils.configs import initial_df_path, path_to_ragas_outputs
from src.utils.functions import *
from src.run_autoprompt import run_autoprompts

## Import and Preprocess Dataset

In [4]:
fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
data = fiqa_eval['baseline']

In [5]:
data.to_pandas()

,question,ground_truths,answer,contexts
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,\nThe best way to deposit a cheque issued to a...,[Just have the associate sign the back and the...
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...
3,Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili...",\nApplying for and receiving business credit c...,[Set up a meeting with the bank that handles y...
4,401k Transfer After Business Closure,[You should probably consult an attorney. Howe...,\nIf your employer has closed and you need to ...,[The time horizon for your 401K/IRA is essenti...
5,What are the ins/outs of writing equipment pur...,[Most items used in business have to be deprec...,\nWriting equipment purchases off as business ...,[You would report it as business income on Sch...
6,Can a entrepreneur hire a self-employed busine...,[Yes. I can by all means start my own company ...,"\nYes, an entrepreneur can hire a self-employe...",[Yes. I can by all means start my own company ...
7,Intentions of Deductible Amount for Small Busi...,"[""If your sole proprietorship losses exceed al...",\nThe intention of deductible amounts for smal...,"[""Short answer, yes. But this is not done thro..."
8,How can I deposit a check made out to my busin...,[You should have a separate business account. ...,\nYou can deposit a check made out to your bus...,"[""I have checked with Bank of America, and the..."
9,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,\nFiling personal taxes with 1099s versus fili...,[Depends whom the 1099 was issued to. If it wa...


### Ragas

This section goes over how to call the Ragas library on the input tealium dataset. Run this section if you want to manually run the initial tealium data (or any updated data) using Ragas. 

In [12]:
# add directory to aws_ragas pyproject.toml file 
!pip install -e ../../aws_ragas/ --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.13 requires botocore==1.34.13, but you have botocore 1.33.9 which is incompatible.
awscli 1.32.13 requires s3transfer<0.11.0,>=0.10.0, but you have s3transfer 0.8.2 which is incompatible.


In [13]:
# For some reason this is not working when I add it to the pyproject.toml, will investigate later
!pip install llama-index==0.9.6.post1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.4/896.4 kB 14.9 MB/s eta 0:00:0000:01


In [14]:
import sys
path_to_ragas_code = '/home/ec2-user/SageMaker/aws_ragas/src'
sys.path.append(path_to_ragas_code)

In [15]:
from ragas.metrics.answer_correctness import AnswerCorrectness, answer_correctness
from ragas.metrics.answer_relevance import AnswerRelevancy, answer_relevancy
from ragas.metrics.answer_similarity import AnswerSimilarity, answer_similarity
from ragas.metrics.context_precision import (
    ContextPrecision,
    ContextRelevancy,
    context_precision,
)
from ragas.metrics.context_recall import ContextRecall, context_recall
from ragas.metrics.critique import AspectCritique
from ragas.metrics.faithfulness import Faithfulness, faithfulness

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/57.4M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/517 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [16]:
metrics = [
    context_recall,
    context_precision,
    answer_relevancy,
    faithfulness,
    answer_similarity,
    answer_correctness
]

In [23]:
from ragas.evaluation import evaluate
# uncomment below line to run the evaluate function, otherwise bring in a
#   dataset that already has ragas outputs. 
result = evaluate(data.select(range(5)), metrics=metrics)

evaluating with [context_recall]


100%|██████████| 1/1 [01:13<00:00, 73.57s/it]


evaluating with [context_precision]


100%|██████████| 1/1 [00:10<00:00, 10.36s/it]


evaluating with [answer_relevancy]


...

evaluating with [answer_similarity]


100%|██████████| 1/1 [00:02<00:00,  2.87s/it]


evaluating with [answer_correctness]


100%|██████████| 1/1 [01:32<00:00, 92.73s/it]


AttributeError: 'Result' object has no attribute 'to_csv'

In [26]:
result.to_pandas().to_csv(path_to_ragas_outputs, index=False)
# result = pd.read_csv('/home/ec2-user/SageMaker/Auto_Prompt/data/ragas_output.csv')
result.to_pandas().head()

,question,contexts,answer,ground_truths,context_recall,context_precision,answer_relevancy,faithfulness,answer_similarity,answer_correctness
0,How to deposit a cheque issued to an associate...,[Just have the associate sign the back and the...,\nThe best way to deposit a cheque issued to a...,[Have the check reissued to the proper payee.J...,0.666667,0.0,0.930311,1.000000,0.524963,0.429148
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...,1.000000,1.0,0.969544,1.000000,0.526689,0.763345
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...,0.714286,0.0,0.874973,1.000000,0.353534,0.476767
3,Applying for and receiving business credit,[Set up a meeting with the bank that handles y...,\nApplying for and receiving business credit c...,"[""I'm afraid the great myth of limited liabili...",1.000000,0.0,0.648423,0.909091,0.772560,0.886280
4,401k Transfer After Business Closure,[The time horizon for your 401K/IRA is essenti...,\nIf your employer has closed and you need to ...,[You should probably consult an attorney. Howe...,0.000000,0.0,0.804983,1.000000,0.646157,0.323079


## Automatic Prompt Generation

Here, automatic prompt generation is done using specific templates (available in instruction_generation_templates folder). 

In [6]:
files = [
    "one-paragraph_instruction_with_demo.txt", 
    "one-sentence_instruction_with_demo.txt", 
    "step-by-step_instruction_with_demo.txt"
]

In [7]:
run_autoprompts(True)

/home/ec2-user/SageMaker/Auto_Prompt/src/run_autoprompt.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_prompts['contexts'] = fix_prompts['contexts'].apply(literal_eval)
/home/ec2-user/SageMaker/Auto_Prompt/src/run_autoprompt.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_prompts['ground_truths'] = fix_prompts['ground_truths'].apply(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call_

9


  0%|                                          | 0/27 [00:00<?, ?it/s]

Gen 1:   Here is a one paragraph instruction on how to solve the task without mentioning any context:

To complete the task, follow these steps in a clear and concise manner. First, carefully examine what is being asked of you to understand the goal or purpose that needs to be accomplished. Second, think
Gen 2:   Here is a one paragraph instruction on how to approach the task without mentioning any specifics:

To successfully complete the task at hand, it is important to first gain a thorough understanding of what is being asked and identify the core issue or problem presented. Look for any clues within the available information that could
----------


  7%|██▌                               | 2/27 [00:04<00:56,  2.25s/it]

...

Draw
Gen 1:   Here is a one sentence instruction on how to solve the task without mentioning any context:

"Methodically consider each component and their interrelations, independently determine the implications of all factors combined through self-guided reasoning alone, then concisely communicate the process."
Gen 2:   Here is a one sentence instruction for the task without mentioning any context:

"Examine the given situation, deduce the most relevant connections and provide an original solution."
----------


100%|█████████████████████████████████| 27/27 [01:51<00:00,  4.18s/it]

Draw
Gen 1:   Here is a step-by-step instruction on how to solve the given task without mentioning any content in the context:

1. Read the task carefully to understand what is being asked.

2. Identify the key pieces of information provided such as any rules, constraints, or goals
Gen 2:   Here is a step-by-step instruction on how to solve the given task without mentioning any content from the context:

1. Read the full details provided in the task carefully to understand what is being asked. 

2. Break down the task into smaller and more manageable steps.
----------


100%|█████████████████████████████████| 27/27 [01:54<00:00,  4.24s/it]

Draw


Below is the output of the best three instructions for each question in the Tealium dataset. 

In [8]:
new_prompts = pd.read_csv('../data/new_prompts.csv')

In [9]:
new_prompts.head()

,question,contexts,answer,ground_truths,context_recall,context_precision,answer_relevancy,faithfulness,answer_similarity,answer_correctness,best_prompt,second_best_prompt,third_best_prompt
0,How to deposit a cheque issued to an associate...,['Just have the associate sign the back and th...,\nThe best way to deposit a cheque issued to a...,"[""Have the check reissued to the proper payee....",0.666667,0.0,0.930311,1.000000,0.524963,0.429148,Carefully analyze both the question and contex...,Thoroughly analyze the question and context in...,Study the question and provided context carefu...
1,1 EIN doing business under multiple business n...,['You\'re confusing a lot of things here. Comp...,"\nYes, it is possible to have one EIN doing bu...","[""You're confusing a lot of things here. Compa...",0.714286,0.0,0.874973,1.000000,0.353534,0.476767,Focus on understanding the core problem descri...,"Objectively consider the question and context,...",Analyze the inputs to identify the key details...
2,Applying for and receiving business credit,['Set up a meeting with the bank that handles ...,\nApplying for and receiving business credit c...,"['""I\'m afraid the great myth of limited liabi...",1.000000,0.0,0.648423,0.909091,0.772560,0.886280,"Analyze the question, context and output caref...","Absorbing the question, context and output is ...","Study the question, context and output thoroug..."
3,401k Transfer After Business Closure,['The time horizon for your 401K/IRA is essent...,\nIf your employer has closed and you need to ...,"[""You should probably consult an attorney. How...",0.000000,0.0,0.804983,1.000000,0.646157,0.323079,Objectively internalize all information specif...,Analyze both the question and context objectiv...,Analyze the question and context provided. Ide...


Below are the prompts/instructions mapped to ids. 

In [10]:
prompt_ids = pd.read_csv('../data/prompt_ids.csv')

In [11]:
prompt_ids.head()

,Prompts,Prompt_Ids
0,Carefully analyze both the question and contex...,0
1,Focus on understanding the core problem descri...,1
2,"Analyze the question, context and output caref...",2
3,Objectively internalize all information specif...,3
4,Thoroughly analyze the question and context in...,4
